In [11]:
import os
import cv2
import imutils
from imutils import face_utils
import dlib
import glob2
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import distance as dist

In [12]:
DATASET = './dmd/binary_labels'
CSV_FILE = 'landmarks_ratios.csv'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('dlib/shape_predictor_68_face_landmarks.dat')

EYE_AR_THRESH = 0.3
MOUTH_AR_THRESH = 0.2

In [13]:
def get_landmarks_ratios(frame):

    frame = imutils.resize(frame, width=640)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces
    faces = detector(gray)
    print(f'Faces: {len(faces)}')

    if len(faces) == 0:
        return None

    # get the largest face
    largest_face = None;
    for face in faces:
        largest_face_area = 0;
        if face.area() > largest_face_area:
            largest_face = face

    shape = predictor(gray, largest_face)

    # Extracting the indices of the facial features
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    (mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["inner_mouth"]

    # Get coordinates for left eye, right eye, and mouth
    left_eye = [(shape.part(i).x, shape.part(i).y) for i in range(lStart, lEnd)]
    right_eye = [(shape.part(i).x, shape.part(i).y) for i in range(rStart, rEnd)]
    mouth = [(shape.part(i).x, shape.part(i).y) for i in range(mStart, mEnd)]

    # Compute aspect ratios for the eyes and mouth
    def eye_aspect_ratio(eye):
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear

    def mouth_aspect_ratio(mouth):
        A = dist.euclidean(mouth[2], mouth[6])
        C = dist.euclidean(mouth[0], mouth[4])
        mar = A / C
        return mar

    left_ear = eye_aspect_ratio(left_eye)
    right_ear = eye_aspect_ratio(right_eye)
    mouth_ar = mouth_aspect_ratio(mouth)

    return left_ear, right_ear, mouth_ar

In [17]:
if os.path.exists(CSV_FILE):
    csv = pd.read_csv
else:
    # Create a DataFrame to hold the data
    df = pd.DataFrame(columns=["Image", "Left_Eye_AR", "Right_Eye_AR", "Mouth_AR", "Awake"])
    file_paths = glob2.glob(DATASET + '/**/*.jpg')
    data = []

    for i, file_path in enumerate(file_paths):

        awake = 1 if 'awake' in file_path else 0
        print(f'Awake: {awake}')
        img = cv2.imread(file_path)
        ratios = get_landmarks_ratios(img)

        if img is not None and ratios is not None:
            # for ratio in ratios:

            print(f"{ratios}\n")
            if img is not None and ratios is not None:
                row = {
                    "Image": file_path,
                    "Left_Eye_AR": ratios[0],
                    "Right_Eye_AR": ratios[1],
                    "Mouth_AR": ratios[2],
                    "Awake": awake
                }
                data.append(row)


    df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)  # concatenate the new data with the old dataframe

    df.to_csv('landmarks_ratios.csv', index=False)



Awake: 0
Faces: 0
Awake: 0
Faces: 1
(0.1800020524176937, 0.2055270768301462, 0.3022438607339301)

Awake: 0
Faces: 1
(0.2805283198220218, 0.2487592975524973, 0.029399051601892736)

Awake: 0
Faces: 1
(0.3078314151048782, 0.2890735821752002, 0.6206896551724138)

Awake: 0
Faces: 1
(0.3820018448736402, 0.3859651178085249, 0.07895830523185818)

Awake: 0
Faces: 1
(0.2631578947368421, 0.26430342040599236, 0.08789381029635715)

Awake: 0
Faces: 0
Awake: 0
Faces: 1
(0.26539634065922857, 0.2028242685634354, 0.03006244442563018)

Awake: 0
Faces: 1
(0.2890735821752002, 0.22800237002164542, 0.8275862068965517)

Awake: 0
Faces: 1
(0.2326803115618832, 0.2381322242764817, 0.03423934786989504)

Awake: 0
Faces: 1
(0.24961508830135312, 0.2102353324910547, 0.06575248692937263)

Awake: 0
Faces: 1
(0.21347047650771073, 0.2111500763274038, 0.0)

Awake: 0
Faces: 1
(0.11043152607484653, 0.12437964877624864, 0.027735009811261455)

Awake: 0
Faces: 1
(0.11694886282802262, 0.17616606585441102, 0.32558139534883723)

